In [14]:
PREDS_PATH = "../predictions/meddoplace-flat-v0-meddoplace-flat/"
GS_PATH = "../meddoplace_flat/test"
PREDS_PATH = PREDS_PATH.rstrip('/')
GS_PATH = GS_PATH.rstrip('/')

In [15]:
import os
import pandas as pd
import csv

In [16]:
preds_files = [filename for filename in os.listdir(PREDS_PATH) if filename.endswith(".ann")]

In [17]:
gs_files = [filename for filename in os.listdir(GS_PATH) if filename.endswith(".ann")]

In [18]:
preds_mentions = pd.DataFrame()
gs_mentions = pd.DataFrame()

In [19]:
for file in gs_files:
    file_df = pd.read_csv(os.path.join(GS_PATH, file), quoting=csv.QUOTE_NONE,
                 sep="\t", usecols=[0, 1, 2], names=['ann_id', "label", 'text'], header=None)
    file_df['start_span'] = file_df['label'].apply(lambda elem: int(elem.split()[1]))
    file_df['end_span'] = file_df['label'].apply(lambda elem: int(elem.split()[2]))
    file_df['label'] = file_df['label'].apply(lambda elem: elem.split()[0])
    file_df['filename'] = file
    gs_mentions = pd.concat([gs_mentions, file_df], ignore_index=True)
gs_mentions = gs_mentions[['filename', 'ann_id', 'label', 'start_span', 'end_span', 'text']]
gs_mentions['mention'] = gs_mentions.apply(lambda annot: annot['filename']+'|'+annot['label']+'|'+str(annot['start_span'])+'|'+str(annot['end_span']), axis=1)

In [20]:
for file in preds_files:
    file_df = pd.read_csv(os.path.join(PREDS_PATH, file), quoting=csv.QUOTE_NONE,
                 sep="\t", usecols=[0, 1, 2], names=['ann_id', "label", 'text'], header=None)
    file_df['start_span'] = file_df['label'].apply(lambda elem: int(elem.split()[1]))
    file_df['end_span'] = file_df['label'].apply(lambda elem: int(elem.split()[2]))
    file_df['label'] = file_df['label'].apply(lambda elem: elem.split()[0])
    file_df['filename'] = file
    preds_mentions = pd.concat([preds_mentions, file_df], ignore_index=True)
preds_mentions = preds_mentions[['filename', 'ann_id', 'label', 'start_span', 'end_span', 'text']]
preds_mentions['mention'] = preds_mentions.apply(lambda annot: annot['filename']+'|'+annot['label']+'|'+str(annot['start_span'])+'|'+str(annot['end_span']), axis=1)

In [21]:
# WRITE FILES TO MEDDOPLACE SCORING SCRIPT FORMAT (TSV)
gs_mentions[['filename', 'ann_id', 'label', 'start_span', 'end_span', 'text']].to_csv(GS_PATH.split('/')[-1]+'.tsv', sep='\t', index=False, header=True)
preds_mentions[['filename', 'ann_id', 'label', 'start_span', 'end_span', 'text']].to_csv(PREDS_PATH.split('/')[-1]+'.tsv', sep='\t', index=False, header=True)

In [57]:
gs_mentions["interval_closed"] = gs_mentions.apply(lambda annot: pd.Interval(annot['start_span'], annot['end_span'], closed='both'), axis=1)
gs_mentions["interval_open"] = gs_mentions.apply(lambda annot: pd.Interval(annot['start_span'], annot['end_span'], closed='neither'), axis=1)
preds_mentions["interval_closed"] = preds_mentions.apply(lambda annot: pd.Interval(annot['start_span'], annot['end_span'], closed='both'), axis=1)
preds_mentions["interval_open"] = preds_mentions.apply(lambda annot: pd.Interval(annot['start_span'], annot['end_span'], closed='neither'), axis=1)
gs_mentions['index'] = gs_mentions.index
preds_mentions['index'] = preds_mentions.index

In [58]:
df_merged = gs_mentions.merge(preds_mentions, on='filename', suffixes=['_gs', '_pred'])

In [59]:
df_merged['exact_match'] = (df_merged['start_span_gs'] == df_merged['start_span_pred']) & (df_merged['end_span_gs'] == df_merged['end_span_pred'])
df_merged['exact_match'].value_counts()

exact_match
False    843
True     153
Name: count, dtype: int64

In [60]:
df_merged[df_merged['exact_match']].query("label_gs != label_pred")

,filename,ann_id_gs,label_gs,start_span_gs,end_span_gs,text_gs,mention_gs,interval_closed_gs,interval_open_gs,index_gs,ann_id_pred,label_pred,start_span_pred,end_span_pred,text_pred,mention_pred,interval_closed_pred,interval_open_pred,index_pred,exact_match
260,cc_ca36.ann,T5,PROFESION,2984,2993,cuidadora,cc_ca36.ann|PROFESION|2984|2993,"[2984, 2993]","(2984, 2993)",56,T6,SITUACION_LABORAL,2984,2993,cuidadora,cc_ca36.ann|SITUACION_LABORAL|2984|2993,"[2984, 2993]","(2984, 2993)",48,True
522,14759.ann,T14,SITUACION_LABORAL,6275,6288,professionals,14759.ann|SITUACION_LABORAL|6275|6288,"[6275, 6288]","(6275, 6288)",81,T12,PROFESION,6275,6288,professionals,14759.ann|PROFESION|6275|6288,"[6275, 6288]","(6275, 6288)",72,True
590,14759.ann,T19,SITUACION_LABORAL,7788,7801,professionals,14759.ann|SITUACION_LABORAL|7788|7801,"[7788, 7801]","(7788, 7801)",85,T16,PROFESION,7788,7801,professionals,14759.ann|PROFESION|7788|7801,"[7788, 7801]","(7788, 7801)",76,True


In [61]:
# X is within Y (nested), and not exact match already
df_merged['pred_shorter'] = df_merged.apply(lambda annot: (annot['start_span_pred'] in annot['interval_closed_gs']) & (annot['end_span_pred'] in annot['interval_closed_gs']) & (not annot['exact_match']), axis=1)
df_merged['pred_longer'] = df_merged.apply(lambda annot: (annot['start_span_gs'] in annot['interval_closed_pred']) & (annot['end_span_gs'] in annot['interval_closed_pred']) & (not annot['exact_match']), axis=1)

In [62]:
# Some overlap
df_merged[df_merged['pred_shorter']][['filename', 'label_gs', 'start_span_gs', 'end_span_gs', 'text_gs', 'label_pred', 'start_span_pred', 'end_span_pred', 'text_pred']].to_csv("fp_fn_analysis/pred_shorter.tsv", sep='\t', index=False, header=True)

In [63]:
# Some overlap
df_merged[df_merged['pred_longer']][['filename', 'label_gs', 'start_span_gs', 'end_span_gs', 'text_gs', 'label_pred', 'start_span_pred', 'end_span_pred', 'text_pred']].to_csv("fp_fn_analysis/pred_longer.tsv", sep='\t', index=False, header=True)

In [64]:
# X starts within Y; AND X ends after Y (usually 0 cases)
df_merged['partial'] = df_merged.apply(lambda annot: 
                                       (annot['start_span_gs'] in annot['interval_open_pred']) & (annot['end_span_gs'] not in annot['interval_closed_pred'])
                                       |
                                       (annot['start_span_pred'] in annot['interval_open_gs']) & (annot['end_span_pred'] not in annot['interval_closed_gs']), axis=1)
df_merged[df_merged['partial']][['filename', 'label_gs', 'start_span_gs', 'end_span_gs', 'text_gs', 'label_pred', 'start_span_pred', 'end_span_pred', 'text_pred']].to_csv("fp_fn_analysis/partial.tsv", sep='\t', index=False, header=True)

In [65]:
df_merged[(df_merged['pred_shorter'] | df_merged['pred_longer'] | df_merged['partial'])].query("label_gs != label_pred")

,filename,ann_id_gs,label_gs,start_span_gs,end_span_gs,text_gs,mention_gs,interval_closed_gs,interval_open_gs,index_gs,...,end_span_pred,text_pred,mention_pred,interval_closed_pred,interval_open_pred,index_pred,exact_match,pred_shorter,pred_longer,partial
759,cc_ca34.ann,T2,PROFESION,97,115,tutor de la classe,cc_ca34.ann|PROFESION|97|115,"[97, 115]","(97, 115)",119,...,102,tutor,cc_ca34.ann|SITUACION_LABORAL|97|102,"[97, 102]","(97, 102)",107,False,True,False,False
960,cc_ca21.ann,T1,PROFESION,1388,1403,cuidadora d’aus,cc_ca21.ann|PROFESION|1388|1403,"[1388, 1403]","(1388, 1403)",167,...,1397,cuidadora,cc_ca21.ann|SITUACION_LABORAL|1388|1397,"[1388, 1397]","(1388, 1397)",156,False,True,False,False


In [66]:
# Most frequent FP
not_spurious = df_merged[(df_merged['exact_match'] | df_merged['pred_shorter'] | df_merged['pred_longer'] | df_merged['partial'])]['mention_pred'].unique()
spurious_mentions = preds_mentions[~preds_mentions['mention'].isin(gs_mentions['mention']) & ~preds_mentions['mention'].isin(not_spurious)]

In [67]:
spurious_mentions[['filename', 'label', 'start_span', 'end_span', 'text']].to_csv("fp_fn_analysis/spurious_mentions.tsv", sep='\t', index=False, header=True)

In [68]:
# Most frequent FN
not_missing = df_merged[(df_merged['exact_match'] | df_merged['pred_shorter'] | df_merged['pred_longer'] | df_merged['partial'])]['mention_gs'].unique()
missing_mentions = gs_mentions[~gs_mentions['mention'].isin(preds_mentions['mention']) & ~gs_mentions['mention'].isin(not_missing)]

In [69]:
missing_mentions[['filename', 'label', 'start_span', 'end_span', 'text']].to_csv("fp_fn_analysis/missing_mentions.tsv", sep='\t', index=False, header=True)

In [70]:
# Most frequent TP
gs_mentions[gs_mentions['mention'].isin(preds_mentions['mention'])]

,filename,ann_id,label,start_span,end_span,text,mention,interval_closed,interval_open,index
2,cc_ca30.ann,T3,PROFESION,2842,2860,metge de capçalera,cc_ca30.ann|PROFESION|2842|2860,"[2842, 2860]","(2842, 2860)",2
5,cc_ca43.ann,T1,PROFESION,488,507,Treballadora Social,cc_ca43.ann|PROFESION|488|507,"[488, 507]","(488, 507)",5
7,cc_ca27.ann,T1,PROFESION,1163,1173,dermatòleg,cc_ca27.ann|PROFESION|1163|1173,"[1163, 1173]","(1163, 1173)",7
8,cc_ca27.ann,T2,PROFESION,1291,1302,dermatòlegs,cc_ca27.ann|PROFESION|1291|1302,"[1291, 1302]","(1291, 1302)",8
9,13068.ann,T1,PROFESION,379,387,pediatre,13068.ann|PROFESION|379|387,"[379, 387]","(379, 387)",9
...,...,...,...,...,...,...,...,...,...,...
184,cc_atencion_primaria_cat70.ann,T1,PROFESION,438,443,pagès,cc_atencion_primaria_cat70.ann|PROFESION|438|443,"[438, 443]","(438, 443)",184
185,9312.ann,T1,PROFESION,1719,1736,otorinolaringòleg,9312.ann|PROFESION|1719|1736,"[1719, 1736]","(1719, 1736)",185
186,14080.ann,T1,PROFESION,897,909,especialista,14080.ann|PROFESION|897|909,"[897, 909]","(897, 909)",186
187,14080.ann,T2,PROFESION,1793,1805,especialista,14080.ann|PROFESION|1793|1805,"[1793, 1805]","(1793, 1805)",187


In [71]:
df_merged[df_merged['pred_shorter']]['text_pred'].apply(lambda text: text.lower()).value_counts().to_csv("fp_fn_analysis/pred_shorter_pred_top.tsv", sep='\t', index=True, header=True)
df_merged[df_merged['pred_shorter']]['text_gs'].apply(lambda text: text.lower()).value_counts().to_csv("fp_fn_analysis/pred_shorter_gs_top.tsv", sep='\t', index=True, header=True)
df_merged[df_merged['pred_longer']]['text_pred'].apply(lambda text: text.lower()).value_counts().to_csv("fp_fn_analysis/pred_longer_pred_top.tsv", sep='\t', index=True, header=True)
df_merged[df_merged['pred_longer']]['text_gs'].apply(lambda text: text.lower()).value_counts().to_csv("fp_fn_analysis/pred_longer_gs_top.tsv", sep='\t', index=True, header=True)
missing_mentions['text'].apply(lambda text: text.lower()).value_counts().to_csv("fp_fn_analysis/missing_mentions_top.tsv", sep='\t', index=True, header=True)
spurious_mentions['text'].apply(lambda text: text.lower()).value_counts().to_csv("fp_fn_analysis/spurious_mentions_top.tsv", sep='\t', index=True, header=True)

In [107]:
# Get TP, FP, FN
df_remaining = df_merged.copy()
TPs_gs = df_remaining[df_remaining['exact_match']]['index_gs'].unique()
TPs_pred = df_remaining[df_remaining['exact_match']]['index_pred'].unique()

In [108]:
df_remaining = df_remaining[~(df_remaining['index_gs'].isin(TPs_gs) | df_remaining['index_pred'].isin(TPs_pred))]

In [109]:
ov_TPs_gs = df_remaining[df_remaining['exact_match'] | df_remaining['pred_shorter'] | df_remaining['pred_longer'] | df_remaining['partial']]['index_gs'].unique()
ov_TPs_pred = df_remaining[df_remaining['exact_match'] | df_remaining['pred_shorter'] | df_remaining['pred_longer'] | df_remaining['partial']]['index_pred'].unique()

In [113]:
df_remaining = df_remaining[~(df_remaining['index_gs'].isin(TPs_gs) | df_remaining['index_pred'].isin(TPs_pred))]

13

,filename,ann_id,label,start_span,end_span,text
0,cc_ca30.ann,T1,SITUACION_LABORAL,864,877,anar a classe
1,cc_ca30.ann,T2,PROFESION,1813,1841,professionals especialitzats
2,cc_ca30.ann,T3,PROFESION,2842,2860,metge de capçalera
3,cc_ca30.ann,T4,PROFESION,3041,3052,farmacèutic
4,cc_ca30.ann,T5,SITUACION_LABORAL,621,635,està estudiant
...,...,...,...,...,...,...
184,cc_atencion_primaria_cat70.ann,T1,PROFESION,438,443,pagès
185,9312.ann,T1,PROFESION,1719,1736,otorinolaringòleg
186,14080.ann,T1,PROFESION,897,909,especialista
187,14080.ann,T2,PROFESION,1793,1805,especialista
